In [1]:
library("metacell")
library("tgstat")
library("Matrix")
library("dplyr")


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
setwd(here::here())

In [3]:
scdb_init("scrna_db_mm10_star/",force_reinit = T)

initializing scdb to scrna_db_mm10_star/



In [4]:
mat_new_name = "embexe_bmp4"

In [6]:
tgconfig::override_params(config_file = "config/emb_exe.yaml",package = "metacell")

In [7]:
mat = scdb_mat(mat_new_name)

In [8]:
min(colSums(mat@mat))

[1] 1500

In [9]:
if(0) {
  bad_genes = read.csv('mc2/embexe_bmp4_og2.lateral_gene_names.csv',h = T)
bad_genes = bad_genes$gene
bad_genes = union(bad_genes,c('Bmp4','H19;Mir675'))

mcell_add_gene_stat(mat_new_name, mat_new_name, force=T)

mcell_gset_filter_varmean(mat_new_name, mat_new_name, T_vm=0.1, force_new=T)
mcell_gset_filter_cov(mat_new_name, mat_new_name, T_tot=50, T_top3=3)

gset = scdb_gset(mat_new_name)
nms = names(gset@gene_set)
#bad gene that will be removed from list of genes that helps to mark metacell 
bad_g = c(grep("^Rpl",nms,v=T),grep("^Gm",nms,v=T),grep("Rps",nms,v=T), "Bmp4")

bad_g = c(bad_g, bad_genes)

gset_f = gset_new_restrict_nms(gset=gset, bad_g, inverse = T, "feat filt")
scdb_add_gset(mat_new_name, gset_f)

mcell_add_cgraph_from_mat_bknn(mat_id = mat_new_name, 
                                 gset_id = mat_new_name, 
                                 graph_id = mat_new_name,
                                 K = 100,
                                 dsamp = T)

  
}


In [1]:
color_query_by_reference = function(cgraph_id,
                                    query_cls_md,
                                    ref_cls_color,
                                    max_knn_cgraph = 50,
                                    knn_color_space = 1,
                                    color_query_cls_with_few_ref_neighbors = T,
                                    threshold_few_ref_neighbors = 0.5) {
  
  cgraph = scdb_cgraph(cgraph_id)
  
  query_cls_f = intersect(names(query_cls_md),cgraph@nodes)
  
  ref_cls = names(ref_cls_color)
  ref_cls_f = intersect(ref_cls,cgraph@nodes)
  all_cls = c(query_cls_f,ref_cls_f)
  
  cls1 = levels(cgraph@edges$mc1)
  cls2 = levels(cgraph@edges$mc2)
  
  # add color and type to 
  cl_to_type = c(query_cls_md[query_cls_f],rep("ref",length(ref_cls_f)))
  names(cl_to_type) = all_cls
  
  type_cls1 = cl_to_type[cls1]
  names(type_cls1) = cls1
  type_cls2 = cl_to_type[cls2]
  names(type_cls2) = cls2
  
  color_cls1 = ref_cls_color[cls1]
  names(color_cls1) = cls1
  color_cls2 = ref_cls_color[cls2]
  names(color_cls2) = cls2
  
  cgraph@edges$type1 = type_cls1[cgraph@edges$mc1]
  cgraph@edges$type2 = type_cls2[cgraph@edges$mc2]  
  cgraph@edges$color1 = color_cls1[cgraph@edges$mc1]
  cgraph@edges$color2 = color_cls2[cgraph@edges$mc2]
  
  
  # report on cells which have few reference neighbors
  cl_vs_type = table(cgraph@edges$mc1,cgraph@edges$type2)
  cl_vs_type = cl_vs_type[all_cls,]
  cls_with_few_neighbors = rownames(cl_vs_type)[rowSums(cl_vs_type) <10]
  cl_vs_type_n = cl_vs_type[rowSums(cl_vs_type) > 0,]
  cl_vs_type_n = cl_vs_type_n/rowSums(cl_vs_type_n)
  
  cls_with_low_fraction_of_ref_cell_neighbors = rownames(cl_vs_type_n)[1 - cl_vs_type_n[,"ref"] > threshold_few_ref_neighbors]
  
  if(!color_query_cls_with_few_ref_neighbors) {
    query_cls_f = setdiff(query_cls_f,cls_with_low_fraction_of_ref_cell_neighbors)
  }
  
  # color annotation of query_cls - restrict to max_knn_cgraph neighbors in the cgraph
  
  f = cgraph@edges$w > (1 - max_knn_cgraph/100)
  graph_f = cgraph@edges[f,]
  
  
  # calculate color contingency table for query vs ref and ref vs ref
  cl_vs_color = table(graph_f$mc1,graph_f$color2)
  cl_vs_color_ref = cl_vs_color[ref_cls_f,]
  cl_vs_color_query = cl_vs_color[query_cls_f,]
  cl_vs_color_ref = cl_vs_color_ref[rowSums(cl_vs_color_ref) > 0,]
  cl_vs_color_query = cl_vs_color_query[rowSums(cl_vs_color_query) > 0,]
  
  ref_ref_col_cor = tgs_cor_knn(x = t(cl_vs_color_ref),y = t(cl_vs_color_ref),knn = (knn_color_space + 1))
  query_ref_col_cor = tgs_cor_knn(x = t(cl_vs_color_query),y = t(cl_vs_color_ref),knn = knn_color_space)
  
  
  sample_nn = sample(2:(knn_color_space + 1),size = nrow(cl_vs_color_ref),replace = T)
  sample_ind = c(0:(nrow(cl_vs_color_ref)-1))*(knn_color_space + 1) + sample_nn
  
  original_cl = ref_ref_col_cor$col1[sample_ind]
  ref_sampled_color = ref_cls_color[as.character(ref_ref_col_cor$col2[sample_ind])]
  ref_original_color = ref_cls_color[as.character(original_cl)]
  names(ref_original_color) = original_cl
  names(ref_sampled_color) = original_cl
  
  
  # sample color for query cls
  sample_nn = sample(1:knn_color_space,size = nrow(cl_vs_color_query),replace = T)
  sample_ind = c(0:(nrow(cl_vs_color_query)-1))*knn_color_space + sample_nn
  
  query_cls_col = ref_cls_color[as.character(query_ref_col_cor$col2[sample_ind])]
  names(query_cls_col) = as.character(query_ref_col_cor$col1[sample_ind])
  
  cmp_col_annot = list(query_cls_col = query_cls_col,
                       cls_with_low_fraction_of_ref_cell_neighbors = cls_with_low_fraction_of_ref_cell_neighbors,
                       ref_cls_orig_color = ref_original_color,
                       ref_cls_sampled_color = ref_sampled_color,
                       cls_with_few_neighbors = cls_with_few_neighbors,
                       max_knn_cgraph = max_knn_cgraph,
                       knn_color_space = knn_color_space,
                       query_cls_col_dist = cl_vs_color_query,
                       ref_cls_col_dist = cl_vs_color_ref,
                       cl_type_dist = cl_vs_type)
  
  return(cmp_col_annot)
}

In [11]:
mat_nm = mat_new_name
ref_mc_id = 'mm_embexe'
cgraph_id = mat_nm
mat = scdb_mat(mat_nm)
mc_wt = scdb_mc(ref_mc_id)
  
query_cells = colnames(mat@mat)[(mat@cell_metadata[colnames(mat@mat),"cell_genotype"] %in% c("Bmp4 Emb unclear","Bmp4 Emb-KO","Bmp4 gKO","Control Tetraploid","WT/Het Bmp4","OG2","empty/ignored"))]
  
ref_cells = names(mc_wt@mc)
  
query_cls_md = mat@cell_metadata[query_cells,"cell_genotype"] 
names(query_cls_md) = query_cells
  
ref_cls_color = mc_wt@colors[mc_wt@mc[ref_cells]]
names(ref_cls_color)  = ref_cells



cmp_annot = color_query_by_reference(cgraph_id = cgraph_id, 
                                     query_cls_md = query_cls_md, 
                                     ref_cls_color = ref_cls_color,
                                     threshold_few_ref_neighbors = 0.9,
                                     max_knn_cgraph = 100)
   


In [12]:
length(cmp_annot$cls_with_low_fraction_of_ref_cell_neighbors)

[1] 171

In [13]:
cells_all = intersect(colnames(mat@mat),names(mc_wt@mc))

In [14]:
egc_wt_old = t(tgs_matrix_tapply(mat@mat[,cells_all],mc_wt@mc[cells_all],sum))
egc_wt_old = t(t(egc_wt_old)/colSums(egc_wt_old))

In [15]:
gset = scdb_gset('embexe')

bad_genes = read.csv('mc2/embexe_bmp4_og2.lateral_gene_names.csv',h = T)
bad_genes = bad_genes$gene
bad_genes = union(bad_genes,c('Bmp4','H19;Mir675'))


feat_genes = setdiff(names(gset@gene_set), bad_genes)

In [16]:
reg = 1e-5
query_ref_cor = tgs_cor(log2(as.matrix(mat@mat[feat_genes,cmp_annot$cls_with_low_fraction_of_ref_cell_neighbors] + 1)),
                        log2(egc_wt_old[feat_genes,] + reg))

In [17]:
best_ref = apply(query_ref_cor,1,which.max)

In [18]:
cmp_annot$query_cls_col[ cmp_annot$cls_with_low_fraction_of_ref_cell_neighbors] = mc_wt@colors[best_ref]

In [19]:
saveRDS(cmp_annot, file=sprintf("data/cmp_annot_%s.RData",mat_nm))

### Transfer time annotations

In [20]:
excluded_colors = mc_wt@color_key$color[39:51]

In [25]:
library("Matrix")
library("qlcMatrix")

Loading required package: slam

Loading required package: sparsesvd



In [26]:
get_query_time_dist = function(query_cls_md,atlas_time,graph_id) {
  
  cgraph = scdb_cgraph(graph_id)
  
  query_cls = intersect(names(query_cls_md),cgraph@nodes)
  atlas_cls = intersect(names(atlas_time),cgraph@nodes)
  atlas_time = atlas_time[atlas_cls]
  query_cls_md = query_cls_md[query_cls]
  
  
  cell_names = c(1:length(cgraph@nodes))
  names(cell_names) = cgraph@nodes
  
  graph = cgraph@edges
  graph$mc1 = as.factor(graph$mc1)
  graph$mc2 = as.factor(graph$mc2)
  levels(graph$mc1) = cell_names[levels(graph$mc1)]
  levels(graph$mc2) = cell_names[levels(graph$mc2)]
  
  # adjacency matrix 
  knn_mat = sparseMatrix(as.numeric(graph$mc1),as.numeric(graph$mc2),x = graph$w)
  colnames(knn_mat) = cgraph@nodes
  rownames(knn_mat) = cgraph@nodes
  
  knn_mat_f = knn_mat[query_cls,atlas_cls]
  
  a = rowMax(X = knn_mat_f,which = T)
  time_match_ind = summary(a$which)
  
  query_time_match = atlas_time[time_match_ind$j]
  names(query_time_match) = query_cls[time_match_ind$i]
  
  query_time_dist = table(query_cls_md[time_match_ind$i],atlas_time[time_match_ind$j])
  tmp = matrix(0,nrow = nrow(query_time_dist),ncol = length(unique(atlas_time)))
  rownames(tmp) = rownames(query_time_dist)
  colnames(tmp) = sort(unique(atlas_time))
  tmp[rownames(query_time_dist),colnames(query_time_dist)] = query_time_dist
  query_time_dist = tmp
  
  
  return(list(query_time_dist = query_time_dist,query_time_match = query_time_match))
}

In [27]:
get_atlas_time_dist = function(query_cls_md,atlas_time,graph_id) {
  
  
  # query_cls_md is a named vector with query_cls as names and the embryo annotation or sth similar as value
  # atlas_time is a named vector giving the atlas_time for each atlas cell
  
  cgraph = scdb_cgraph(graph_id)
  
  atlas_cls = intersect(names(atlas_time),cgraph@nodes)
  atlas_time = atlas_time[atlas_cls]
  
  
  cell_names = c(1:length(cgraph@nodes))
  names(cell_names) = cgraph@nodes
  
  graph = cgraph@edges
  graph$mc1 = as.factor(graph$mc1)
  graph$mc2 = as.factor(graph$mc2)
  levels(graph$mc1) = cell_names[levels(graph$mc1)]
  levels(graph$mc2) = cell_names[levels(graph$mc2)]
  
  knn_mat = sparseMatrix(as.numeric(graph$mc1),as.numeric(graph$mc2),x = graph$w)
  colnames(knn_mat) = cgraph@nodes
  rownames(knn_mat) = cgraph@nodes
  
  knn_mat_f = knn_mat[atlas_cls,atlas_cls]
  for(i in unique(atlas_time)) {
    f = atlas_time == i
    knn_mat_f[f,f] = 0
  }
  
  a = rowMax(X = knn_mat_f,which = T)
  time_match_ind = summary(a$which)
  
  atlas_time_match = atlas_time[time_match_ind$j]
  names(atlas_time_match) = atlas_cls[time_match_ind$i]
  
  q_time_match_tmp = table(atlas_time[time_match_ind$i], atlas_time_match)
  q_time_match = matrix(0,nrow = length(unique(atlas_time)),ncol = length(unique(atlas_time)))
  rownames(q_time_match) = sort(unique(atlas_time))
  colnames(q_time_match) = sort(unique(atlas_time))
  q_time_match[rownames(q_time_match_tmp),colnames(q_time_match_tmp)] = q_time_match_tmp
  
  return(list(atlas_time_dist = q_time_match,atlas_time_match = atlas_time_match))
}

In [28]:
time_dist_best_match = function(query_time_dist,atlas_time_dist) {
  
  rank_to_time = read.table(file = "data/embexe.transcriptional_rank_developmental_time.tsv",stringsAsFactors = F,h = T,sep = "\t")
  dev_time = rank_to_time$developmental_time
  
  query_ref_cor = cor(t(as.matrix(as.data.frame.matrix(query_time_dist))),
                          t(as.matrix(as.data.frame.matrix(atlas_time_dist))))
  
  
  wt_dev_time = rank_to_time$developmental_time[as.numeric(colnames(query_ref_cor))] 

  best_fit = apply(query_ref_cor,1,function(emb_cor) {
    smooth_spline = smooth.spline(x = wt_dev_time,y = emb_cor,spar = 0.8)
    y = smooth_spline$y
    return(which.max(y))
  })
  
  best_fit = as.numeric(colnames(query_ref_cor))[best_fit]
  return(best_fit)
}

In [29]:
cgraph_id =  mat_nm
mat = scdb_mat(mat_nm)
mc_wt = scdb_mc("embexe_recolored")
  
  # Exclude extraembryonic ectoderm and parietal endoderm
  
  cmp_annot = readRDS(file = sprintf("data/cmp_annot_%s.RData",mat_nm))
  
  f = mat@cell_metadata[colnames(mat@mat),"cell_genotype"] %in% c("Bmp4 Emb unclear","Bmp4 Emb-KO","Bmp4 gKO","Control Tetraploid","WT/Het Bmp4")
  query_cells = colnames(mat@mat)[f]
  query_cells = intersect(query_cells,names(cmp_annot$query_cls_col))
  
  if(!is.null(excluded_colors)) {
    excluded_cls = query_cells[cmp_annot$query_cls_col[query_cells] %in% excluded_colors]
    query_cells = setdiff(query_cells,excluded_cls)
  } else {
    message("Did not exclude any cell types for timing the embryos.")
  }
  
  n_cells_per_embryo = table(mat@cell_metadata[query_cells,"embryo"])
  
  query_embryos = names(n_cells_per_embryo)[n_cells_per_embryo > 10]
  
  
  wt_cls = intersect(names(mc_wt@mc)[ !(mc_wt@colors[mc_wt@mc] %in% excluded_colors) ],colnames(mat@mat))
  wt_cls = wt_cls[!is.na(mat@cell_metadata[wt_cls,"transcriptional_rank"])]
  atlas_time = mat@cell_metadata[wt_cls,"transcriptional_rank"]
  names(atlas_time) = wt_cls
  
  data_dir = sprintf("data/%s",mat_nm)
  if(!dir.exists(data_dir)) {
    dir.create(data_dir)
  }
  
  data_dir = sprintf("data/%s/time_match",mat_nm)
  if(!dir.exists(data_dir)) {
    dir.create(data_dir)
  }
  
  
  # first get atlas time distribution
  
  atlas_time_dist = get_atlas_time_dist(atlas_time = atlas_time,graph_id = cgraph_id)
  
  # first timing using control cells only
  query_cls = query_cells[( mat@cell_metadata[query_cells,"cell_genotype"] %in%  c("Bmp4 Emb unclear","Bmp4 Emb-KO","Bmp4 gKO","Control Tetraploid","WT/Het Bmp4") ) & ( mat@cell_metadata[query_cells,"embryo"] %in%  query_embryos )]
  query_cls_md = mat@cell_metadata[query_cls,"embryo"]
  names(query_cls_md) = query_cls
  
  query_time_dist= get_query_time_dist(query_cls_md = query_cls_md,atlas_time = atlas_time,graph_id = cgraph_id)
  
  time_dist_query = list(atlas_time_dist = atlas_time_dist$atlas_time_dist,
                        query = query_time_dist$query_time_dist)
  
  #saveRDS(time_dist_query,file = sprintf("%s/time_dist_query.RData",data_dir))
  
  chim_emb_summary = as.data.frame.matrix(table(mat@cell_metadata[query_cells,"embryo"],mat@cell_metadata[query_cells,"cell_type"]))
  chim_emb_summary$embryo = rownames(chim_emb_summary)
  chim_emb_summary$best_rank_query = NA
  chim_emb_summary[rownames(time_dist_query$query),"best_rank_query"] = time_dist_best_match(atlas_time_dist = time_dist_query$atlas_time_dist,
                                                                                       query_time_dist = time_dist_query$query)
  
  #write.table(chim_emb_summary,file = sprintf("%s/time_match_summary.txt",data_dir),sep ="\t",row.names = F)